In [ ]:
import sys
import os
from os import listdir
from statistics import mean,median
from alive_progress import alive_bar
from pdfminer.high_level import extract_text
from pdfrw import PdfReader, PdfWriter, PageMerge

In [ ]:
# Function to split files based on character per line in .pdf
# Splits on mean and average of character count
# Run this for the .pdf files in /data

def split_files():
    directory = "data"

    try:
        os.mkdir("split")
    except:
        pass

    def splitpage(src):
        """Split a page into two (left and right)"""
        # Yield a result for each half of the page
        for x_pos in (0, 0.5):
            yield PageMerge().add(src, viewrect=(x_pos, 0, 0.5, 1)).render()

    split = 0
    y=0
    already_done=os.listdir('split')
    print (len(already_done))
    dirs=os.listdir(directory)
    for dir in dirs:
        paths=os.listdir(f"{directory}/{dir}")
        for document in paths :
            y+=1
            print (y,end='\r')
            if document[-4:]=='.pdf' and document not in already_done:
                text = extract_text(f"{directory}/{dir}/{document}")
                x=[]
                #print (text)
                for i in text.splitlines():

                    # Filter out table data
                    if (len(i)>40):

                        x.append(len(i))
                # Calculate the average length of sentences outside tables
                try: 
                    res_mean=mean(x)
                except:
                    res_mean=100
                try:
                    res_median=median(x)
                except:
                    res_median=100
                    print ('Error on file:',document,'\n')
                text = text.splitlines()

                if len(max(text, key=len)) <=67 or len(max(text, key=len)) >67 and res_mean <55 and res_median<55:
                    #print ('Splitting ',document,end='\r')
                    
                    writer = PdfWriter()
                    page_number = 0
                    for page in PdfReader(f"{directory}/{dir}/{document}").pages:
                        writer.addpages(splitpage(page))
                    writer.write(f"split/{document}")
                    split += 1
  
    print(f"Split pdfs: {split}")

In [ ]:
split_files()

# Outputs to /split

In [ ]:
# Output files from the above function splits files in pages sequentially, so they have alternating language per page
# This function makes 2 .pdfs from this file in the corresponding language

path = "split"

try:
    os.mkdir("Nederlands")
except:
    pass

try:
    os.mkdir("Francais")
except:
    pass

for document in os.listdir(path):
    writer_NL = PdfWriter()
    writer_FR = PdfWriter()
    page_number = 0
    for page in PdfReader(f"{path}/{document}").pages: 
        page_number += 1
        if page_number%2 == 1:
            writer_NL.addpage(page)
        else:
            writer_FR.addpage(page)
    writer_NL.write(f"Nederlands/NL_{document}")
    writer_FR.write(f"Francais/FR_{document}")